In [11]:
from flask import Flask, render_template, send_file
import psycopg2
import pandas as pd
import plotly.express as px
import scipy.stats as st
import plotly 
import plotly.utils 
import json
import numpy as np

In [2]:
conn_string = "host='localhost' dbname='phone_usage_db' user='postgres' password='postgres'"
conn = psycopg2.connect(conn_string)

print("Database opened successfully")
cursor = conn.cursor();
query = 'select * from phone_user_behavior'
cursor.execute(query)

colnames = [desc[0] for desc in cursor.description]


print(colnames)


phone_df = pd.DataFrame(cursor.fetchall(), columns=colnames)

conn.close()

print("Database Closed successfully")

Database opened successfully
['user_id', 'device_model', 'operating_system', 'app_usage_minutes_per_day', 'screen_time_hours_per_day', 'battery_drain_mah_per_day', 'number_of_apps_installed', 'data_usage_mb_per_day', 'age', 'gender', 'user_behavior_class']
Database Closed successfully


In [3]:
phone_df['age'].sort_values()

561    18
116    18
685    18
686    18
258    18
       ..
225    59
192    59
660    59
619    59
597    59
Name: age, Length: 700, dtype: int64

In [4]:
json_data = phone_df.to_json(orient='index')

In [13]:
  phone_df_battery = phone_df.copy()
bins = [18, 25, 35, 50]
labels = ["18-25", "25-35", "35-50"]
phone_df_battery['age_groups'] = pd.cut(phone_df_battery['age'], bins, labels=labels)
# Calculating mean battery drain for each age group
mean_battery_age = phone_df_battery.groupby(["age_groups"])["battery_drain_mah_per_day"].mean()
mean_battery_age_df = pd.DataFrame({"age_groups": mean_battery_age.index, "average_battery_drain": mean_battery_age.values})
# Creating a bar chart using Plotly
fig = px.bar(
    mean_battery_age_df,
    x="age_groups",
    y="average_battery_drain",
    title="Average Battery Drain Per Age Group",
    labels={"age_groups": "Age Groups", "average_battery_drain": "Battery Drain (mah/day)"},
    text_auto=True
)
# Customizing the figure's appearance
fig.update_layout(
    xaxis_title="Age Groups",
    yaxis_title="Battery Drain (mah/day)",
    yaxis_range=[0, mean_battery_age.max() + 500],
    template="plotly_white"
)

graph_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

print(graph_json)

{"data": [{"alignmentgroup": "True", "hovertemplate": "Age Groups=%{x}<br>Battery Drain (mah/day)=%{y}<extra></extra>", "legendgroup": "", "marker": {"color": "#636efa", "pattern": {"shape": ""}}, "name": "", "offsetgroup": "", "orientation": "v", "showlegend": false, "textposition": "auto", "texttemplate": "%{y}", "x": ["18-25", "25-35", "35-50"], "xaxis": "x", "y": [1629.1083333333333, 1465.034090909091, 1500.1068376068376], "yaxis": "y", "type": "bar"}], "layout": {"template": {"data": {"barpolar": [{"marker": {"line": {"color": "white", "width": 0.5}, "pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "barpolar"}], "bar": [{"error_x": {"color": "#2a3f5f"}, "error_y": {"color": "#2a3f5f"}, "marker": {"line": {"color": "white", "width": 0.5}, "pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "bar"}], "carpet": [{"aaxis": {"endlinecolor": "#2a3f5f", "gridcolor": "#C8D4E3", "linecolor": "#C8D4E3", "minorgridcolor": "#C8D4E3", "startlinecol

/var/folders/c5/49gjf4515dq14r1sfkhgdnv00000gn/T/ipykernel_15665/3701107589.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

